In [2]:
import tensorflow as tf
import numpy as np
import scipy.io

In [3]:
vgg19 = scipy.io.loadmat('/Users/leoadlakha/Documents/Research Work/Image Qualty Enhancement/Our Work/Code/imagenet-vgg-verydeep-19.mat')

In [4]:
vgg19

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sat Sep 19 12:27:40 2015',
 '__version__': '1.0',
 '__globals__': [],
 'layers': array([[array([[(array([[array([[[[ 0.39416704, -0.08419707, -0.03631314, ..., -0.10720515,
           -0.03804016,  0.04690642],
          [ 0.46418372,  0.03355668,  0.10245045, ..., -0.06945956,
           -0.04020201,  0.04048637],
          [ 0.34119523,  0.09563112,  0.0177449 , ..., -0.11436455,
           -0.05099866, -0.00299793]],
 
         [[ 0.37740308, -0.07876257, -0.04775979, ..., -0.11827433,
           -0.19008617, -0.01889699],
          [ 0.41810837,  0.05260524,  0.09755926, ..., -0.09385028,
           -0.20492788, -0.0573062 ],
          [ 0.33999205,  0.13363543,  0.02129423, ..., -0.13025227,
           -0.16508926, -0.06969624]],
 
         [[-0.04594866, -0.11583115, -0.14462094, ..., -0.12290562,
           -0.35782176, -0.27979308],
          [-0.04806903, -0.00658076, -0.02234544, ..., -0.0878844 ,
           -0

In [5]:
vgg19layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

In [6]:
weights = vgg19['layers'][0]

In [7]:
def vgg19Network() :
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        layer_type = name[:4]
        if layer_type == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif layer_type == 'relu':
            current = tf.nn.relu(current)
        elif layer_type == 'pool':
            current = _pool_layer(current)
        net[name] = current

    return net

In [8]:
IMAGE_MEAN = np.array([123.68 ,  116.779,  103.939])

def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1), padding='SAME')
    return tf.nn.bias_add(conv, bias)

def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME')

def preprocess(image):
    return image - IMAGE_MEAN